**Implementing a two-layer perceptron.** In this coding session, we will get some practice implementing a two-layer perceptron. The code in this session is heavily borrowed from the [Machine Learning with PyTorch and Scikit-learn book's Github repo](https://github.com/rasbt/machine-learning-book/tree/main).

**Notation for neural networks.** At the end of last class, we introduced the following notation for a neural network: let $(V, E, w)$ be a directed, weighted graph, and suppose the vertex set $V$ can be split into disjoint sets of vertices, $V = \bigcup_{\ell = 0}^L V_\ell$ such that if $e \in E$ then $e = (v, v')$ for $v \in V_{\ell}$ and $v' \in V_{\ell + 1}$ for some $\ell = 0, 1, \ldots, L - 1\}$. The number $L$ is called the number of *layers* of the neural network. The integers $d_{\rm in} := |V_0|$ and $d_{\rm out} := |V_L|$ are called the *input* and *output* dimensions of the network. The parameter $w$ are the *weights*, which we can think of either as a function $w \colon E \to \mathbb{R}$, or as a collection of matrices $(W_1, \ldots, W_{L})$ where each $W_\ell \in \mathbb{R}^{|V_{\ell}| \times |V_{\ell - 1}|}$, along with biases $b_1, \ldots, b_L$ such that $b_\ell \in \mathbb{R}^{|V_{\ell}|}$.

We assume as given a non-linearity $\sigma \colon \mathbb{R} \to \mathbb{R}$, for example a ReLU unit ($\sigma(\tau) := \max(0, \tau)$), a logistic unit $\sigma(\tau) = (1 + e^{-\tau})$ or something else.

The network then generates output via a *forward pass*. Given an input $x \in \mathbb{R}^{d_{\rm in}}$, we calculate the output $o_0(x) = (x_1, \ldots, x_d, 1)$, and then for subsequent layers we put $a_\ell(x) = W_\ell o_{\ell - 1}(x) + b_\ell$ and $o_{\ell}(x) := \sigma(a_{\ell}(x))$ for output. The output of the network is, finally, $o_L(x)$.

**Specializing to a two-layer perceptron.** In this lab, we will get into the nitty gritty details of a two-layer, fully connected, perceptron.

**Importing and preparing the MNIST dataset** We will use the MNIST dataset, which is a collection of images of handwritten digits (1, 2, ... , 9) half written by high school students and half written by employees of the US Census Bureau.

In [ ]:
import numpy as np

from sklearn.datasets import fetch_openml

X, y = fetch_openml('mnist_784', version=1, return_X_y=True, parser='auto')
X = X.values
y = y.astype(int).values

print(X.shape)
print(y.shape)

The images in the MNIST dataset consist of 28×28 pixels, and each pixel is represented by a grayscale intensity value. Here, fetch_openml already unrolled the 28×28 pixels into one-dimensional row vectors, which are the rows in our X array (784 per row or image) above. The second array (y) returned by the fetch_openml function contains the corresponding target variable, the class labels (integers 0-9) of the handwritten digits.

Next, let’s normalize the pixels values in MNIST to the range –1 to 1 (originally 0 to 255):

In [ ]:
X = (X/255. - .5)*2

We'll be using gradient descent based methods to train out model, and for these methods it is especially important that the features are normalized. Now let's visualize the first elements of each class.

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(nrows=2, ncols=5, sharex=True, sharey=True)
ax = ax.flatten()
for i in range(10):
    img = X[y == i][0].reshape(28, 28)
    ax[i].imshow(img, cmap='Greys')

ax[0].set_xticks([])
ax[0].set_yticks([])
plt.tight_layout()
#plt.savefig('figures/11_4.png', dpi=300)
plt.show()

Now let's plot different 7s:

In [ ]:
fig, ax = plt.subplots(nrows=5, ncols=5, sharex=True, sharey=True)
ax = ax.flatten()
for i in range(25):
    img = X[y == 7][i].reshape(28, 28)
    ax[i].imshow(img, cmap='Greys')

ax[0].set_xticks([])
ax[0].set_yticks([])
plt.tight_layout()
# plt.savefig('figures/11_5.png', dpi=300)
plt.show()

Finally, we will split the dataset into training, validation, and test sets.

In [ ]:
from sklearn.model_selection import train_test_split

# Split the code into a train, validation, and test set of sizes
# 55000, 5000, and 10000 respectively. Make sure you stratify on the
# labels vector. (If you aren't sure how to do this, take a look at
# the code from previous coding sessions.)

### YOUR CODE HERE ###


**Building a basic two-layer perceptron class.** We'll now work on implementing a basic two-layer neural network model. To this end, we'll need to handle a detail involving the encoding of the class labels. As they currently stand, the class labels are integers in $[9]$, but to work with them in a neural network model, we'll need to use an encoding to the output dimension. The encoding we'll use is known as a *one-hot* encoding, where the label $i \in [9]$ is mapped to the $i + 1$st basis vector in $\mathbb{R}^{10}$. The code for this encoding follows.

In [ ]:
def int_to_onehot(y, num_labels):
    ary = np.zeros((y.shape[0], num_labels))
    for i, val in enumerate(y):
        ary[i, val] = 1

    return ary

We'll also use a sigmoid activation for our network.

In [ ]:
# note that the sigmoid method takes effect elementwise when applied to a numpy array
def sigmoid(z):                                        
    return 1. / (1. + np.exp(-z))

We can now implement our 2 layer perceptron. Because it has two layers, its constructor will accept three integers, each corresponding to the number of nodes at each layer. An important detail to observe is that when we initialize the weights of our perceptron, we initialize them *randomly*. For now, we will ignore the details of the backpropogation which is necessary for training, and you will implement part of the initialization method as well as part of the forward propogation method.

In [ ]:
class NeuralNetMLP:
    def __init__(self, num_features, num_hidden, num_classes, random_seed=123):
        super().__init__()
        
        self.num_classes = num_classes
        
        # hidden
        rng = np.random.RandomState(random_seed)

        # Initialize first weight matrix
        self.weight_in = rng.normal(
            loc=0.0, scale=0.1, size=(num_hidden, num_features))
        self.bias_in = np.zeros(num_hidden)
        
        # Initialize second weight matrix in the same way, but with the right
        # size
        self.weight_out = rng.normal(
            loc=0.0, scale=0.1, size=### YOUR CODE HERE ###)
        self.bias_out = np.zeros(### YOUR CODE HERE###)
        
    def forward(self, x):
        # Hidden layer
        # input dim: [n_examples, n_features] dot [n_hidden, n_features].T
        # output dim: [n_examples, n_hidden]
        a_h = np.dot(x, self.weight_in.T) + self.bias_in
        o_h = sigmoid(a_h)

        # Output layer
        # input dim: [n_examples, n_hidden] dot [n_classes, n_hidden].T
        # output dim: [n_examples, n_classes]
        a_out = ### YOUR CODE HERE ###
        o_out = ### YOUR CODE HERE ###
        return o_h, o_out

    def backward(self, x, o_h, o_out, y):  
    
        #########################
        ### Output layer weights
        #########################
        
        # onehot encoding
        y_onehot = int_to_onehot(y, self.num_classes)

        # Part 1: dLoss/dOutWeights
        ## = dLoss/dOutAct * dOutAct/dOutNet * dOutNet/dOutWeight
        ## where DeltaOut = dLoss/dOutAct * dOutAct/dOutNet
        ## for convenient re-use
        
        # input/output dim: [n_examples, n_classes]
        d_loss__d_o_out = 2.*(o_out - y_onehot) / y.shape[0]

        # input/output dim: [n_examples, n_classes]
        d_o_out__d_z_out = o_out * (1. - o_out) # sigmoid derivative

        # output dim: [n_examples, n_classes]
        delta_out = d_loss__d_o_out * d_o_out__d_z_out # "delta (rule) placeholder"

        # gradient for output weights
        
        # [n_examples, n_hidden]
        d_z_out__dw_out = o_h
        
        # input dim: [n_classes, n_examples] dot [n_examples, n_hidden]
        # output dim: [n_classes, n_hidden]
        d_loss__dw_out = np.dot(delta_out.T, d_z_out__dw_out)
        d_loss__db_out = np.sum(delta_out, axis=0)
        

        #################################        
        # Part 2: dLoss/dHiddenWeights
        ## = DeltaOut * dOutNet/dHiddenAct * dHiddenAct/dHiddenNet * dHiddenNet/dWeight
        
        # [n_classes, n_hidden]
        d_z_out__o_h = self.weight_out
        
        # output dim: [n_examples, n_hidden]
        d_loss__o_h = np.dot(delta_out, d_z_out__o_h)
        
        # [n_examples, n_hidden]
        d_o_h__d_z_h = o_h * (1. - o_h) # sigmoid derivative
        
        # [n_examples, n_features]
        d_z_h__d_w_h = x
        
        # output dim: [n_hidden, n_features]
        d_loss__d_w_h = np.dot((d_loss__o_h * d_o_h__d_z_h).T, d_z_h__d_w_h)
        d_loss__d_b_h = np.sum((d_loss__o_h * d_o_h__d_z_h), axis=0)

        return (d_loss__dw_out, d_loss__db_out, 
                d_loss__d_w_h, d_loss__d_b_h)

Let's initialize our model; we'll make the hidden layer have 50 units.

In [ ]:
model = NeuralNetMLP(num_features=28*28, num_hidden=50, num_classes=10)

**Reminders on GD, SGD, and mini-batch SGD.** Now, we'll a form of gradient descent to train out model. Here's a brief reminder on gradient descent methods (we'll go into more detail in lectures).

Suppose we want to solve the minimization problem
$$
\min_{\theta \in \mathbb{R}^d} f(\theta).
$$ The function $f$ is called the *objective*. Then gradient descent uses the iteration, for some initial point $\theta_0$,
$$
\theta_{t + 1} = \theta_t - \eta_t \nabla f(\theta_t),
$$ where $\eta_t \in \mathbb{R}$ is a *learning rate* that is specified by the user (typically a constant in the case of gradient descent proper).

An objective $f$ that arises frequently in statistics and machine learning is of the form
$$
f(\theta) = \frac1n \sum_{i = 1}^n \ell(h_{\theta}(x_i), y_i),
$$ where $\ell$ is a loss function and $(h_{\theta})_{\theta \in \Theta}$ is a parametrized model class. Indeed, this objective corresponds to an *empirical loss*. For example, in the setting of two layer perceptrons we'll use $\ell(z , y) := \| z - y \|^2$ the $\ell_2$-loss, and $h_{\theta}$ the neural network with parameters $\theta = (W_1, W_2, b_1, b_2)$, and the sum will be over the training data of MNIST. Often (and in our case), the $n$ will be very large, making direct computation of $\nabla f(\theta)$ impractical and thus ruling out the gradient descent algorithm.

A work-around for this issue, which has additional attractive properties that we won't discuss now, is called *stochastic gradient descent (SGD)*. Here, we take a uniformly random $i \in [n]$, and use it to form a *stochastic gradient*
$$
\hat \nabla f(\theta) := \nabla_\theta \ell(h_{\theta}(x_i), y_i).
$$ Observe that, over the randomness of this uniform random sample, $\mathbb{E}[\hat \nabla f(\theta)] = \nabla f(\theta)$. It is thus plausible that we can use this stochastic gradient as a replacement for the full gradient $\nabla f(\theta)$, and run a stochastic version of the gradient descent algorithm. This is called *stochastic gradient descent*, and uses the updates
$$
\theta_{t + 1} = \theta_t - \eta_t \hat \nabla f(\theta_t),
$$ where, importantly, we use fresh randomness at each iteration to form the stochastic gradient $\hat \nabla f(\theta_t)$.

Finally, the SGD algorithm can be excessively noisy due to the fact that we use only one data point to form the stochastic gradient $\hat \nabla f(\theta_t)$. This is remedied by averaging over several stochastic gradients to produce *mini-batch SGD*. Specifically, let $M$ be a fixed positive integer (the mini-batch size) and let $\{i_1, \ldots, i_M\} \subset [n]$ be a random subset of $M$ integers in $[n]$. Then we can form the averaged stochastic gradient
$$
\bar \nabla^M f(\theta) := \frac1M \sum_{m = 1 }^M \nabla_\theta \ell(h_{\theta}(x_{i_m}), y_{i_m})
$$ The minibatch SGD algorithm is then
$$
\theta_{t + 1} = \theta_t - \eta_t \bar \nabla^M f(\theta_t).
$$ In practice, the way the randomness for the minibatch data or SGD is typically generated is by first randomly permuting the training data, and then iterating through the permuted data until there is no more data left, and then re-permuting the data and starting over again. Each iteration through the training data is referred to as an *epoch*.

The following utility function (no need to read carefully) generates minibatches appropriate for our use.

In [ ]:
num_epochs = 50
minibatch_size = 100

def minibatch_generator(X, y, minibatch_size):
    indices = np.arange(X.shape[0])
    np.random.shuffle(indices)

    for start_idx in range(0, indices.shape[0] - minibatch_size 
                           + 1, minibatch_size):
        batch_idx = indices[start_idx:start_idx + minibatch_size]
        yield X[batch_idx], y[batch_idx]

Let's now define functions that compute the MSE and accuracy of our model, and check that they give reasonable numbers when run on our existing model (remember, we haven't trained it yet so it is merely randomly initialized).

In [ ]:
def mse_loss(targets, probas, num_labels=10):
    onehot_targets = int_to_onehot(targets, num_labels=num_labels)
    return np.mean((onehot_targets - probas)**2)

def accuracy(targets, predicted_labels):
    return np.mean(predicted_labels == targets) 

_, probas = model.forward(X_valid)
mse = mse_loss(y_valid, probas)

predicted_labels = np.argmax(probas, axis=1)
acc = accuracy(y_valid, predicted_labels)

print(f'Initial validation MSE: {mse:.1f}')
print(f'Initial validation accuracy: {acc*100:.1f}%')

This is a plausible accuracy (about 1/10) since we initialized the data randomly.

Now, we'll use the minibatch method to implement a version of the above methods that will scale to very large data sets where we might otherwise run into memory issue. There's no need to look at this method in detail.

In [ ]:
def compute_mse_and_acc(nnet, X, y, num_labels=10, minibatch_size=100):
    mse, correct_pred, num_examples = 0., 0, 0
    minibatch_gen = minibatch_generator(X, y, minibatch_size)
        
    for i, (features, targets) in enumerate(minibatch_gen):

        _, probas = nnet.forward(features)
        predicted_labels = np.argmax(probas, axis=1)
        
        onehot_targets = int_to_onehot(targets, num_labels=num_labels)
        loss = np.mean((onehot_targets - probas)**2)
        correct_pred += (predicted_labels == targets).sum()
        
        num_examples += targets.shape[0]
        mse += loss

    mse = mse/(i+1)
    acc = correct_pred/num_examples
    return mse, acc

We'll get the same result as before using this large-scale version of the previous methods:

In [ ]:
mse, acc = compute_mse_and_acc(model, X_valid, y_valid)
print(f'Initial valid MSE: {mse:.1f}')
print(f'Initial valid accuracy: {acc*100:.1f}%')

We can now, finally, implement the training method! Note that we will be keeping track of the train and validation MSE and accuracy throughout the optimization so we can make plots below. 

In [ ]:
def train(model, X_train, y_train, X_valid, y_valid, num_epochs,
          learning_rate=0.1):
    
    epoch_loss = []
    epoch_train_acc = []
    epoch_valid_acc = []
    
    for e in range(num_epochs):

        # iterate over minibatches
        minibatch_gen = minibatch_generator(
            X_train, y_train, minibatch_size)

        for X_train_mini, y_train_mini in minibatch_gen:
            
            #### Compute outputs ####
            o_h, o_out = model.forward(X_train_mini)

            #### Compute gradients ####
            d_loss__d_w_out, d_loss__d_b_out, d_loss__d_w_h, d_loss__d_b_h = \
                model.backward(X_train_mini, o_h, o_out, y_train_mini)

            # Each of the d_loss__d_w_out, d_loss__d_b_out, d_loss__d_w_h, d_loss__d_b_h
            # represent the gradient of the loss with respect to the parameters
            # w_out, b_out, w_h, b_h, averaged over the minibatch
            # use them to update the weights here usign the mini-batch SGD formula

            #### Update weights ####
            model.weight_in -= ### YOUR CODE HERE ###
            model.bias_in -= ### YOUR CODE HERE ###
            model.weight_out -= ### YOUR CODE HERE ###
            model.bias_out -= ### YOUR CODE HERE ###
        
        #### Epoch Logging ####        
        train_mse, train_acc = compute_mse_and_acc(model, X_train, y_train)
        valid_mse, valid_acc = compute_mse_and_acc(model, X_valid, y_valid)
        train_acc, valid_acc = train_acc*100, valid_acc*100
        epoch_train_acc.append(train_acc)
        epoch_valid_acc.append(valid_acc)
        epoch_loss.append(train_mse)
        print(f'Epoch: {e+1:03d}/{num_epochs:03d} '
              f'| Train MSE: {train_mse:.2f} '
              f'| Train Acc: {train_acc:.2f}% '
              f'| Valid Acc: {valid_acc:.2f}%')

    return epoch_loss, epoch_train_acc, epoch_valid_acc

Let's see what it does.

In [ ]:
np.random.seed(123) # for the training set shuffling

epoch_loss, epoch_train_acc, epoch_valid_acc = train(
    model, X_train, y_train, X_valid, y_valid,
    num_epochs=50, learning_rate=0.1)

In [ ]:
plt.plot(range(len(epoch_loss)), epoch_loss)
plt.ylabel('Mean squared error')
plt.xlabel('Epoch')
#plt.savefig('figures/11_07.png', dpi=300)
plt.show()

Now, create a plot of the Accuracy vs Epochs of both the training and validation accuracy.

In [ ]:
### YOUR CODE HERE ###
plt.ylabel('Accuracy')
plt.xlabel('Epochs')
plt.legend(loc='lower right')
#plt.savefig('figures/11_08.png', dpi=300)
plt.show()

We finally look at the ultimate test accuracy of the trained model, and plot some of the misclassified digits.

In [ ]:
test_mse, test_acc = compute_mse_and_acc(model, X_test, y_test)
print(f'Test accuracy: {test_acc*100:.2f}%')

In [ ]:
X_test_subset = X_test[:1000, :]
y_test_subset = y_test[:1000]

_, probas = model.forward(X_test_subset)
test_pred = np.argmax(probas, axis=1)

misclassified_images = X_test_subset[y_test_subset != test_pred][:25]
misclassified_labels = test_pred[y_test_subset != test_pred][:25]
correct_labels = y_test_subset[y_test_subset != test_pred][:25]

fig, ax = plt.subplots(nrows=5, ncols=5, 
                       sharex=True, sharey=True, figsize=(8, 8))
ax = ax.flatten()
for i in range(25):
    img = misclassified_images[i].reshape(28, 28)
    ax[i].imshow(img, cmap='Greys', interpolation='nearest')
    ax[i].set_title(f'{i+1}) '
                    f'True: {correct_labels[i]}\n'
                    f' Predicted: {misclassified_labels[i]}')

ax[0].set_xticks([])
ax[0].set_yticks([])
plt.tight_layout()
#plt.savefig('figures/11_09.png', dpi=300)
plt.show()